Reproduce the ["R from Python" plot](https://github.com/WinVector/Examples/blob/main/calling_R_from_Python/plot_from_R_example.ipynb) in Python using plotnine. This is a line for line translation of [significance_power_visuals.R](https://github.com/WinVector/Examples/blob/main/calling_R_from_Python/significance_power_visuals.R).

[Jupyter Widgets documentation](https://ipywidgets.readthedocs.io/en/latest/index.html)

In [58]:
# import our modules
import numpy as np
import pandas as pd
from IPython.display import display, clear_output
from scipy.stats import norm
import matplotlib.pyplot as plt
from data_algebra.cdata import RecordSpecification
from plotnine import (
    aes, 
    ggplot, 
    geom_line, geom_point, geom_ribbon, geom_vline,
    scale_color_manual, scale_fill_manual,
    xlab, ylab,
    ggtitle,
    theme
)

import ipywidgets as widgets

In [2]:
# our parameters of interest
n = 557
r = 0.1
t = 0.061576 # the correct threshold
power = 0.9
significance = 0.02

In [6]:
thresh = t

In [7]:
def make_graph(thresh):
    # convert to what were the function arguments
    stdev = np.sqrt(0.5 / n)
    effect_size = r
    threshold = thresh
    title='Area under the tails give you significance and (1-power)'
    subtitle = 'Significance: control right tail; (1-Power): treatment left tail'
    eps=1e-6
    control_color='#d95f02'
    treatment_color='#1b9e77'

    # define the wide plotting data
    x = set(np.arange(-5 * stdev, 5 * stdev + effect_size, step=stdev / 100))
    x.update([threshold, threshold-eps, threshold+eps])
    x = sorted(x)

    pframe = pd.DataFrame({
        'x': x,
        'control': norm.pdf(x, loc=0, scale=stdev),
        'treatment': norm.pdf(x, loc=effect_size, scale=stdev),
    })
    # control's right tail
    pframe['control_tail'] = np.where(pframe['x'] > threshold, pframe['control'], 0)
    # treatment's left tail
    pframe['treatment_tail'] = np.where(pframe['x'] <= threshold, pframe['treatment'], 0)

    pframe

    # convert from to long for for plotting using the data algebra

    # specify the cdata record transform
    record_transform = RecordSpecification(
        pd.DataFrame({
            'group': ['treatment', 'control'],
            'y': ['treatment', 'control'],
            'tail': ['treatment_tail', 'control_tail'],
        }),
        record_keys=['x'],
        control_table_keys=['group'],
    ).map_from_rows()


    record_transform

    # apply the record transform
    pframelong = record_transform(pframe)

    pframelong

    # make the plot using the plotnine implementation 
    # of Leland Wilkinson's Grammar of Graphics
    # (nearly call equiv to Hadley Wickham ggplot2 realization)
    palette = {'control': control_color, 'treatment': treatment_color}
    p = (
        ggplot(pframelong, aes(x='x', y='y'))
            + geom_line(aes(color='group'))
            + geom_vline(xintercept=threshold, 
                        color='#7f7f7f', 
                        # linewidth=1,
                        )
            + geom_ribbon(aes(ymin=0, ymax='tail', fill='group'), alpha = 0.5)
            + scale_color_manual(values=palette)
            + scale_fill_manual(values=palette)
            + theme(legend_position='none')
            + ylab('density')
            + xlab('observed difference')
            + ggtitle(
                title 
                + "\n" + subtitle,
                )
        )

    return p

In [59]:

w = widgets.FloatSlider(
    value=t,
    min=0.0,
    max=r,
    step=0.001,
    description='Threshold Value:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='.4f'
)

display(w)

button = widgets.Button(description = "Plot it")
output = widgets.Output()

display(button, output)

# this works, need a clear command
def on_button_clicked(b):
    with output:
       clear_output(wait=False)
       print(f'threshold = {w.value}')
       print(make_graph(w.value))
       plt.show()

button.on_click(on_button_clicked)

FloatSlider(value=0.061576, continuous_update=False, description='Threshold Value:', max=0.1, readout_format='…

Button(description='Plot it', style=ButtonStyle())

Output()